In [12]:
import pyspark
from pyspark.sql import SparkSession
import os


def get_spark_session():
  NESSIE_URI = "http://nessie:19120/api/v1"
  MINIO_HOST = 'http://minio:9000'
  MINIO_ACCESS_KEY = "pTo0VEtqzA7hTKsCwCOV"
  MINIO_SECRET_KEY = "B7j2Pt93vVDkYhJ9BsKBi43ZhQtMZU5iP4UznW92"

  jar_packages = [
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1",
    "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.95.0",
    "software.amazon.awssdk:bundle:2.17.178",
    "software.amazon.awssdk:url-connection-client:2.17.178"]

  spark_extensions = [
    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"]

  conf = (
    pyspark.SparkConf()
      .setAppName('iceberg_hello_world')
      .set('spark.jars.packages', ','.join(jar_packages))
      .set('spark.sql.extensions', ','.join(spark_extensions))
      .set('spark.sql.catalog.nessie', "org.apache.iceberg.spark.SparkCatalog")
      .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
      .set('spark.sql.catalog.nessie.ref', 'main')
      .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
      .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
      .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
      .set('spark.sql.catalog.nessie.s3.endpoint', MINIO_HOST)
      .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
      .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
      .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
      .set('spark.hadoop.fs.s3a.path.style.access', 'true')
      .set('spark.hadoop.fs.s3a.connection.timeout', '600000')
      .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
      .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
      .set('spark.hadoop.fs.s3a.endpoint', MINIO_HOST))
  return SparkSession.builder.config(conf=conf).master("spark://spark-master:7077").getOrCreate()



spark = get_spark_session()
spark.sql("SHOW TABLES IN nessie").show()
spark.sql("SHOW DATABASES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |open_2023|      false|
+---------+---------+-----------+

+---------+
|namespace|
+---------+
|  default|
+---------+



In [13]:
## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open_2023")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS nessie.open_2023 USING iceberg AS SELECT * FROM csv_open_2023").show()
spark.sql("SELECT COUNT(*) FROM nessie.open_2023").show()

++
||
++
++

+--------+
|count(1)|
+--------+
|  302231|
+--------+

